In [22]:
import os
import pypdf
import requests
from bs4 import BeautifulSoup
import re

def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename



In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Create a new instance of the Chrome driver
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.mc.mil/Cases/MC-Cases")

xpath = "/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[1]/div/div/div[2]/div[2]/div[1]/div[1]/span/div/button"
# path to KSM et al. case

xpath_filters = "/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[2]/div/div[2]/div/button"

xpath_transcripts = "/html/body/form/div[3]/main/div[1]/div/div/div/div[1]/div[1]/main/div/div[3]/div/div/div[2]/div/div/div[2]/div/div/div[1]/div[3]/label/span"
# checkbox to select transcripts

button = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH, xpath))
    )
button.click()

pager = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.NAME, "pageSelector"))
)

# Scrape all the links on the page ending in .pdf

# Advance to next page


In [20]:
pager.get_attribute("length")

'641'

In [12]:
driver.quit()

In [24]:
# url = "https://www.mc.mil/Cases/MC-Cases"
# the difficulty is that the urls of the pdfs I want are being dynamically exposed via javascript.
# page = requests.get(url)
# soup = BeautifulSoup(page.content, 'html.parser')

# print(soup)

In [ ]:
testpdf = pypdf.PdfReader
page = testpdf.pages[0]
print(page.extract_text())